In [ ]:
%%capture
!pip install langchain openai langchain-experimental duckduckgo-search wikipedia

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


## Tree of Thought Prompting (ToT) Overview

Tree of Thought (ToT) prompting was introduced independently by two groups of researchers in May 2023.

More famously is the Google DeepMind and Princeton group led by Shunyu Yao. They published a paper titled [Tree of Thoughts: Deliberate Problem Solving with Large Language Models](https://arxiv.org/pdf/2305.10601.pdf), and also Jieyi Long from Theta Labs in the paper titled [Large Language Model Guided Tree-of-Thought](https://arxiv.org/pdf/2305.08291.pdf)


You can dig into the code from Shunyu Yao's group on [GitHub](https://github.com/princeton-nlp/tree-of-thought-llm/tree/master) to see how it's implemented in raw Python code.


## 🤖 **1. What's Tree of Thought Prompting?**


- 🌳 **Concept**: ToT is like a brainy framework for smart language models. It helps them solve tricky problems by thinking in a tree-like way, just like we do when we're puzzled!

- 🧠 **Inspiration**: It's all about mimicking how our minds tackle tough tasks—exploring, trying different things, and backtracking if we hit a dead end.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2305.10601/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>Schematic illustrating various approaches to problem solving with LLMs. Each rectangle box represents a thought, which is a coherent language sequence that serves as an intermediate step toward problem solving.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2305.10601">Image Source</a>
</figure>


## 🔍 **2. How Tree of Thought Prompting Functions**

- 🌲 **Tree Layout**: In ToT, each node is a mini thought or step towards cracking the problem.

- 🤔 **Self-Check**: The language model can judge how close these mini thoughts are to solving the puzzle.

- 🚀 **Clever Search Strategies**: It mixes the model's thought-creating skills with search tactics like BFS and DFS. This combo lets it plan ahead and backtrack to find the best answers.

- 🛠️ **Extra Tools**: Some versions have cool add-ons, like a checker to confirm solutions, a memory bank for convo history, and a controller to guide the thought journey.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2305.10601/assets/x5.png" alt="Image Description" style="width:100%">
  <figcaption>A step of deliberate search in a randomly picked Creative Writing task. Given the input, the LM samples 5 different plans, then votes 5 times to decide which plan is best. The majority choice is used to consequently write the output passage with the same sample-vote procedure.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2305.10601">Image Source</a>
</figure>



# 🌲 **3. What Makes Tree of Thought Prompting Stand Out?**

- 📏 **Old-School Prompting**: Traditional methods are like a straight line—ask a question, get an answer. But they don't really wander around or think ahead.

- 🔗 **Chain-of-Thought Style**: This is a bit like building a solution step-by-step. It's more advanced than the straight line, but still doesn't explore different paths.

- 🌳 **ToT's Special Sauce**: ToT is the adventurer of the group. It thinks in multiple directions, can backtrack if a path doesn't work out, and tries different ways to solve a problem.

- 🎮 **Teaming up with Reinforcement Learning**: In some setups, ToT pairs up with reinforcement learning. This means it gets even smarter at exploring and finding solutions.

💡 **In a Nutshell**: ToT is like the maze-runner of prompting techniques. It's all about exploring far and wide, learning from dead ends, and using smart strategies to tackle complex problems. Way different from just going from A to B!



# Let's see ToT in action using LangChain's Experimental library

You can find the code [on GitHub](https://github.com/langchain-ai/langchain/tree/master/libs/experimental/langchain_experimental/tot)



In [ ]:
sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"

problem_description = f"""You are expert level Sudoku player working on a 4x4 Sudoku Puzzle.

The puzzle is delimited by triple backticks.

Here is your puzzle: ```{sudoku_puzzle}```

Instructions:
- Cells marked with '*' need to be filled.
- Use digits 1-4 to fill the cells.
- Each row is separated by the '|' character.
- Ensure no duplicate digits in any row, column, or 2x2 subgrid.
- Retain the known digits from previous valid thoughts.
- Your response can be a step towards the solution or the complete solution itself.
"""

print(problem_description)

You are expert level Sudokue plater working on a 4x4 Sudoku Puzzle.

The puzzle is delimited by triple backticks.

Here is your puzzle: ```3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1```

Instructions:
- Cells marked with '*' need to be filled.
- Use digits 1-4 to fill the cells.
- Each row is separated by the '|' character.
- Ensure no duplicate digits in any row, column, or 2x2 subgrid.
- Retain the known digits from previous valid thoughts.
- Your response can be a step towards the solution or the complete solution itself.



🌲 **Exploring `ToTChecker` Class**

- `ToTChecker` is a blueprint for evaluating thoughts in a "Tree of Thought" format.

🔍 **Abstract Nature of `ToTChecker`**
- Can't be used directly; needs subclasses with implemented methods.

🎮 **Sudoku Example for Implementation**
- Implement your logic (like rule-based or AI) in subclasses for specific scenarios like Sudoku.

In [ ]:
from langchain_experimental.tot import prompts as tot_prompts

# 🔨 **Understanding `Jinja2` Template Syntax**

- Jinja2 uses `{% ... %}` for loops and conditionals, and `{{ ... }}` for printing expressions.

🌱 **Dynamic Content with Jinja2**
- Templates dynamically generate text using input variables.

🌳 **Tree of Thoughts Framework in COT_PROMPT**
- Describes an AI agent generating thoughts. Inputs: `problem_description` and `thoughts`.
- Lists provided `thoughts` under "THOUGHTS" section if any are given.

In [ ]:
print(tot_prompts.COT_PROMPT.template)

You are an intelligent agent that is generating one thought at a time in
a tree of thoughts setting.

PROBLEM 

{{problem_description}}

{% if thoughts %}
THOUGHTS

{% for thought in thoughts %}
{{ thought }}
{% endfor %}
{% endif %}

Let's think step by step.


#### 2. 🤔 **Understanding `PROPOSE_PROMPT` Template**

- Focuses on generating a list of possible valid thoughts.

📝 **Markdown & JSON Format**
- Outputs thoughts as a markdown code snippet, formatted as a JSON list of strings.

📊 **Input Variables**
- Accepts `problem_description`, `thoughts`, and `n` (for number of thoughts to generate).


In [ ]:
print(tot_prompts.PROPOSE_PROMPT.template)

You are an intelligent agent that is generating thoughts in a tree of
thoughts setting.

The output should be a markdown code snippet formatted as a JSON list of
strings, including the leading and trailing "```json" and "```":

```json
[
    "<thought-1>",
    "<thought-2>",
    "<thought-3>"
]
```

PROBLEM

{{ problem_description }}

{% if thoughts %}
VALID THOUGHTS

{% for thought in thoughts %}
{{ thought }}
{% endfor %}

Possible next {{ n }} valid thoughts based on the last valid thought:
{% else %}

Possible next {{ n }} valid thoughts based on the PROBLEM:
{%- endif -%}


#### 3. 🔍 **Exploring `CHECKER_PROMPT` Template**
- Designed to validate thoughts of another AI agent.

💡 **Validation Responses**
- Agent responds with: VALID, INVALID, or INTERMEDIATE after evaluating thoughts.

📖 **Scenario-based Evaluation**
- Presents scenarios for the agent to assess the thoughts' relevance or accuracy.

In [ ]:
print(tot_prompts.CHECKER_PROMPT.template)

You are an intelligent agent, validating thoughts of another intelligent agent.

PROBLEM 

{problem_description}

THOUGHTS

{thoughts}

Evaluate the thoughts and respond with one word.

- Respond VALID if the last thought is a valid final solution to the
problem.
- Respond INVALID if the last thought is invalid.
- Respond INTERMEDIATE if the last thought is valid but not the final
solution to the problem.

This chain of thoughts is


In [ ]:
from typing import Tuple
from langchain.llms import OpenAI
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity
import re

class MyChecker(ToTChecker):
    """
    Custom checker class to evaluate the validity of thoughts against a given Sudoku solution.

    This class inherits from the ToTChecker and provides a specific implementation for evaluating
    Sudoku solutions.
    """

    def evaluate(self, problem_description: str, thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        """
        Evaluate the validity of the thought against the Sudoku solution.

        Args:
            problem_description (str): The description of the Sudoku problem.
            thoughts (Tuple[str, ...]): A tuple of thoughts, where each thought represents a potential solution.

        Returns:
            ThoughtValidity: The validity status of the thought (VALID_FINAL, VALID_INTERMEDIATE, INVALID).
        """

        # Extract the last thought from the tuple
        last_thought = thoughts[-1]

        # Remove spaces and double quotes from the thought to get a clean solution
        clean_solution = last_thought.replace(" ", "").replace('"', "")

        # Convert the clean solution into a regex pattern for matching
        # Replace '*' with '.' to match any digit and escape the '|' character
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")

        # Check if the clean solution matches the exact Sudoku solution
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        # Check if the regex solution matches any part of the Sudoku solution
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        # If neither of the above conditions are met, the thought is invalid
        else:
            return ThoughtValidity.INVALID

In [ ]:
checker = MyChecker()

🧩 **Testing `MyChecker` with Sudoku Thoughts**

- Assertions in Python verify if conditions are met.

🚨 **Purpose of Assertions**
- Check if `MyChecker` evaluates Sudoku thoughts correctly.

✅ **Expected Outcomes**
- The thought `"3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"` should be marked as `VALID_INTERMEDIATE`.
- Represents a partially filled Sudoku board, indicating a step towards a solution.

In [ ]:
assert checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE

🎯 **Evaluating Complete Sudoku Solution**
- Assertion tests if `"3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"` is `VALID_FINAL`.

🧩 **Understanding Sudoku Finality**
- Represents a fully filled Sudoku board, indicating a complete solution.


In [ ]:
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",)) == ThoughtValidity.VALID_FINAL

🔍 **Assessing Nearly Complete Sudoku Board**
- Tests if `"3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1"` is categorized as `VALID_INTERMEDIATE`.

🧩 **Sudoku Board with One Blank Cell**
- Indicates an almost complete puzzle, representing an intermediate step towards the solution.

In [ ]:
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE

This line checks if the thought `"3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1"` is evaluated as `INVALID` by the checker.

The thought represents a Sudoku board that has an error (the numbers 3 and 4 are swapped in the last row), so it's expected to be invalid.

In [ ]:
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",)) == ThoughtValidity.INVALID

#🌳 **Understanding `ToTChain` Class**

- Implements the Tree of Thought methodology with language models and checkers.

###🔧 **Key Attributes of `ToTChain`**

- `k`: Sets the maximum number of conversation rounds.

- `c`: Determines the number of child nodes to explore in each thought node.

In [ ]:
from langchain_experimental.tot.base import ToTChain

llm = OpenAI()

tot_chain = ToTChain(llm=llm,
                     checker=MyChecker(),
                     k=30,
                     c=5,
                     verbose=True,
                     verbose_llm=True
                     )

tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.


> Entering new ProposePromptStrategy chain...
Prompt after formatting:
You are an intelligent agent that is generating thoughts in a tree of
thoughts setting.

The output should be a markdown code snippet formatted as a JSON list of
strings, including the leading and trailing "```json" and "```":

```json
[
    "<thought-1>",
    "<thought-2>",
    "<thought-3>"
]
```

PROBLEM

You are expert level Sudokue plater working on a 4x4 Sudoku Puzzle.

The puzzle is delimited by triple backticks.

Here is your puzzle: ```3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1```

Instructions:
- Cells marked with '*' need to be filled.
- Use digits 1-4 to fill the cells.
- Each row is separated by the '|' character.
- Ensure no duplicate digits in any row, column, or 2x2 subgrid.
- Retain the known digits from previous valid thoughts.
- Your response can be a step towards the solution or the complete solution itself.




Possible next 5 valid tho

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Thought: 3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1

> Finished chain.


'3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'